<a href="https://colab.research.google.com/github/phutaekwondo/CS114.L21/blob/main/duDoanDiemCuoiKy_homeWork/draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

># **Data Discovery**

---

##**Nhóm thực hiện**
1. Lại Nguyễn Vĩnh Phú - 19522020
2. Nguyễn Chí Cường - 19521299
3. Trương Thế Trương - 19522447

##***Dữ liệu***

1. [We code](https://github.com/phutaekwondo/CS114.L21/blob/main/duDoanDiemCuoiKy_homeWork/wecode.csv)
2. [score training](https://github.com/phutaekwondo/CS114.L21/blob/main/duDoanDiemCuoiKy_homeWork/score.csv)

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
!git clone https://github.com/phutaekwondo/CS114.L21

Cloning into 'CS114.L21'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 148 (delta 54), reused 36 (delta 9), pack-reused 0
Receiving objects: 100% (148/148), 9.08 MiB | 16.17 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [3]:
url_wecode = '/content/CS114.L21/duDoanDiemCuoiKy_homeWork/wecode.csv'
url_score = '/content/CS114.L21/duDoanDiemCuoiKy_homeWork/score.csv'

In [4]:
data_wecode = pd.read_csv(url_wecode)
data_score = pd.read_csv(url_score)

In [5]:
data_wecode

,id,assignment_id,problem_id,user_id,is_final,status,pre_score,coefficient,file_name,language_id,created_at,updated_at
0,4,0,3,4,1,SCORE,10000,100,solution-0,2,2020-10-30 10:58:11,2020-10-30 10:58:19
1,5,0,4,4,1,SCORE,10000,100,solution-1,2,2020-10-30 11:07:09,2020-10-30 11:07:17
2,6,0,5,4,0,Compilation Error,0,100,solution-2,2,2020-10-30 11:08:01,2020-10-30 11:23:36
3,7,0,5,4,0,Compilation Error,0,100,solution-3,2,2020-10-30 11:11:41,2020-10-30 11:11:42
4,8,0,5,4,0,Compilation Error,0,100,solution-4,2,2020-10-30 11:15:06,2020-10-30 11:15:07
...,...,...,...,...,...,...,...,...,...,...,...,...
264601,266667,0,25,1796,0,SCORE,0,100,solution-3977,4,2021-04-28 9:34:20,2021-04-28 9:34:27
264602,266668,0,25,1796,0,Compilation Error,0,100,solution-3978,2,2021-04-28 9:35:07,2021-04-28 9:35:08
264603,266669,0,25,1796,1,SCORE,10000,100,solution-3979,4,2021-04-28 9:35:15,2021-04-28 9:35:21
264604,266670,0,150,255,1,SCORE,6250,100,solution-3980,2,2021-05-11 1:34:57,2021-05-11 1:35:28


##Khám phá mục score

In [ ]:
discove_score = {'is_final': data_wecode['is_final'], 'pre_score': data_wecode['pre_score']}
discove_score = pd.DataFrame(discove_score, columns=['is_final', 'pre_score'])

In [ ]:
count_score = Counter(zip(discove_score['is_final'],discove_score['pre_score'])).items()
show_score = pd.DataFrame(count_score, columns=['x', 'y'])
show_score

,x,y
0,"(1, 10000)",95332
1,"(0, 0)",79796
2,"(0, 8000)",7976
3,"(0, 10000)",4508
4,"(1, 0)",2726
...,...,...
656,"(0, 3583)",1
657,"(1, 7250)",1
658,"(0, 5583)",1
659,"(0, 5666)",1


##Tạo dataframe số lượng user_id kết với problem_id

features:

+ số lần nộp bài để pass(theo problem_id, user_id )
+ time last submit - time first submit ( theo pro_id, user_id)
+ max (pre_score) (theo pro_id, user_id)
+ is_final = max(data_wecode['is_final']

features table

user_id | problem_id | số lần nộp | is_final | max(pre_score)|time|
--------|------------|------------|------|----------------|--------|

In [10]:
first_feature = {'user_id': data_wecode['user_id'], 'problem_id': data_wecode['problem_id']}
first_feature = pd.DataFrame(first_feature, columns = ['user_id', 'problem_id'])

#Đếm submit theo user_id và problem_id
count_submit = Counter(zip(first_feature['user_id'], first_feature['problem_id'])).items()
user_feature = pd.DataFrame(count_submit, columns=['user_id, problem_id', 'amount_submit'])#lúc này có user_id và problem_id chung 1 column

#sắp xếp lại data - tách user_id và problem_id
temp = pd.DataFrame(user_feature['user_id, problem_id'].tolist(), columns = ['user_id','problem_id']) #tạo temp = 2 cột user_id và problem_id khi tác ra
del user_feature['user_id, problem_id'] #Xóa cột
user_feature['user_id'], user_feature['problem_id'] = temp['user_id'], temp['problem_id']#thêm lại 2 cột
cols = user_feature.columns.tolist()#sếp xếp lại
cols = cols[-2:] + cols[:1]
user_feature = user_feature[cols]
#sort theo user_id_problem_id và set index
user_feature = user_feature.sort_values(by=['user_id', 'problem_id']).set_index(pd.Index(range(len(user_feature))))

>Lấy max điểm, time, is_final cho từng user_problem_id

In [11]:
#Lấy các features pre_score, is_final, time_max
other_feature_user = {'user_problem_id': pd.Series(zip(data_wecode['user_id'], data_wecode['problem_id'])), 'is_final': data_wecode['is_final'],
                      'pre_score': data_wecode['pre_score'], 'diffirence_time': data_wecode['updated_at']}
other_feature_user = pd.DataFrame(other_feature_user, columns=['user_problem_id', 'is_final', 'pre_score', 'diffirence_time'])
other_feature_user = other_feature_user.sort_values(by='user_problem_id').set_index(pd.Index(range(len(other_feature_user))))
other_feature = other_feature_user.groupby(['user_problem_id'], as_index=False).agg('max')

In [12]:
#Lấy time_min, sau đó tạo thành diffirence_time với time_max với công thức diffirence_time = tim_max - time_min
time_min = {'user_problem_id': pd.Series(zip(data_wecode['user_id'], data_wecode['problem_id'])), 'time_min': data_wecode['updated_at']}
time_min = pd.DataFrame(time_min, columns=['user_problem_id', 'time_min'])
time_min = time_min.sort_values(by='user_problem_id').set_index(pd.Index(range(len(time_min))))
time_min = time_min.groupby(['user_problem_id'], as_index=False).agg('min')
other_feature['diffirence_time'] = pd.to_datetime(other_feature['diffirence_time']) - pd.to_datetime(time_min['time_min'])
other_feature

,user_problem_id,is_final,pre_score,diffirence_time
0,"(1, 81)",1,0,0 days 00:00:00
1,"(1, 370)",1,10000,0 days 00:00:00
2,"(4, 3)",1,10000,0 days 00:00:00
3,"(4, 4)",1,10000,0 days 00:00:00
4,"(4, 5)",1,10000,0 days 00:11:54
...,...,...,...,...
97185,"(1868, 637)",1,10000,1 days 02:20:53
97186,"(1868, 638)",1,10000,1 days 01:39:31
97187,"(1868, 639)",1,10000,1 days 01:22:57
97188,"(1868, 640)",1,10000,1 days 00:43:16


In [57]:
def covert_time(time):
  day_to_seconds = time.days * 24 * 60 * 60
  return day_to_seconds + time.seconds
user_feature['is_final'], user_feature['pre_score'] = other_feature['is_final'], other_feature['pre_score']
user_feature['diffirence_time'] = other_feature['diffirence_time'].apply(covert_time)
user_feature

,user_id,problem_id,amount_submit,is_final,pre_score,diffirence_time
0,1,81,1,1,0,0
1,1,370,1,1,10000,0
2,4,3,1,1,10000,0
3,4,4,1,1,10000,0
4,4,5,5,1,10000,714
...,...,...,...,...,...,...
97185,1868,637,4,1,10000,94853
97186,1868,638,4,1,10000,92371
97187,1868,639,2,1,10000,91377
97188,1868,640,2,1,10000,88996


In [58]:
user_feature.to_csv("/content/CS114.L21/duDoanDiemCuoiKy_homeWork/user_feature.csv")

##Continute

In [60]:
url_feature = '/content/CS114.L21/duDoanDiemCuoiKy_homeWork/user_feature.csv'
user_feature = pd.read_csv('/content/CS114.L21/duDoanDiemCuoiKy_homeWork/user_feature.csv')
user_feature

,Unnamed: 0,user_id,problem_id,amount_submit,is_final,pre_score,diffirence_time
0,0,1,81,1,1,0,0
1,1,1,370,1,1,10000,0
2,2,4,3,1,1,10000,0
3,3,4,4,1,1,10000,0
4,4,4,5,5,1,10000,714
...,...,...,...,...,...,...,...
97185,97185,1868,637,4,1,10000,94853
97186,97186,1868,638,4,1,10000,92371
97187,97187,1868,639,2,1,10000,91377
97188,97188,1868,640,2,1,10000,88996


##Xử lý user_features

> Biến đổi về theo user_id để có thể ánh xạ với feature_score


In [ ]:
synthetic = user_feature.groupby(['user_id']).agg(list)

In [ ]:
synthetic

,problem_id,amount_submit,is_final,pre_score,diffirence_time
user_id,,,,,
1,"[81, 370]","[1, 1]","[1, 1]","[0, 10000]","[0 days 00:00:00, 0 days 00:00:00]"
4,"[3, 4, 5, 6, 7, 8, 9, 81, 101, 104, 106, 107, ...","[1, 1, 5, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10000, 10000, 10000, 10000, 10000, 10000, 100...","[0 days 00:00:00, 0 days 00:00:00, 0 days 00:1..."
5,"[1, 4, 81, 176, 243]","[1, 1, 1, 1, 2]","[1, 1, 1, 1, 1]","[10000, 10000, 0, 10000, 0]","[0 days 00:00:00, 0 days 00:00:00, 0 days 00:0..."
6,"[1, 3, 9, 80, 81, 82, 83, 84, 85, 86, 87, 88, ...","[2, 2, 1, 5, 2, 1, 4, 1, 1, 1, 4, 3, 3, 2, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 10000, 0, 10000, 10000, 10000, 10000, 1000...","[0 days 00:01:45, 0 days 00:00:00, 0 days 00:0..."
7,"[265, 390]","[2, 3]","[1, 1]","[8571, 638]","[0 days 00:09:58, 0 days 00:04:43]"
...,...,...,...,...,...
1860,"[3, 4, 5, 7, 8, 9, 11, 25, 174, 175, 176, 177,...","[1, 2, 2, 3, 2, 1, 2, 2, 1, 2, 2, 4, 2, 1, 2, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10000, 0, 10000, 0, 10000, 0, 0, 10000, 0, 10...","[0 days 00:00:00, 0 days 00:01:55, 0 days 00:0..."
1862,"[1, 10, 11, 176, 181, 182]","[4, 2, 3, 1, 1, 1]","[1, 1, 1, 1, 1, 1]","[10000, 0, 0, 10000, 10000, 10000]","[0 days 00:36:28, 0 days 00:01:53, 0 days 00:0..."
1864,"[1, 3, 4, 5, 6, 7, 8, 10, 11, 80, 81, 82, 84, ...","[5, 6, 10, 1, 3, 5, 5, 3, 12, 4, 5, 3, 2, 1, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10000, 10000, 10000, 10000, 0, 10000, 10000, ...","[1 days 20:42:59, 1 days 20:43:20, 1 days 20:4..."


>Bảng điểm cho problem_id

In [8]:
#tried_feature
count_sb_problem = Counter(data_wecode['problem_id']).items()
score_table = pd.DataFrame(count_sb_problem, columns=['problem_id', 'tried'])
score_table = score_table.sort_values('problem_id')
score_table = score_table.set_index('problem_id')
#solved_feature
solved_feature = {'problem_id': data_wecode['problem_id'], 'solved': data_wecode['is_final']}
solved_feature = pd.DataFrame(solved_feature, columns=['problem_id', 'solved'])
solved_feature = solved_feature.groupby(['problem_id']).agg('sum')

In [9]:
score_table['solved'] = solved_feature['solved']
score_table['Tried/Solved'] = score_table['tried']/solved_feature['solved']
score_table = score_table.replace(max(score_table['Tried/Solved']), 0) #thay những Tried/Solved = number/0 thành 0. Vì không có ai giải được cũng không tính điểm thôi
score_table

,tried,solved,Tried/Solved
problem_id,,,
1,3249,1001,3.245754
2,231,53,4.358491
3,2992,988,3.028340
4,4213,899,4.686318
5,1271,840,1.513095
...,...,...,...
692,5,2,2.500000
695,29,8,3.625000
696,20,3,6.666667


Idea: từ DataFrame **synthetic** và **score_table** tạo feature bằng cách tính điểm

Vd: user_id = 1 giải problem 81 và 370 chiếu qua DataFrame score_table lấy Tried/Solved của problem 81 và 370 cộng lại.

Hoặc có thể dùng môt số công thức khác.

#**Score Data**

In [ ]:
features_score = {'id': data_score['id'], 'CK': data_score['CK']}
features_score = pd.DataFrame(features_score, columns = ['id', 'CK'])

In [ ]:
features_score

,id,CK
0,22,8.5
1,24,9.5
2,27,10
3,28,9.5
4,29,8.5
...,...,...
970,1857,4
971,1859,1.5
972,1862,7.5
973,1865,7.5


# **Nhận Xét**